### Imports

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.extend(['/home/nkinsky/Documents/GitHub/NeuroPy'])
sys.path.extend(['/home/nkinsky/Documents/GitHubPrivate/pythonprogs/DataPaths/'])

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects
from stats_utils import (
    bootstrap_resample,
    get_bootstrap_prob,
    get_bootstrap_prob_paired,
)

### Firing rate bootstrap means

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks
drop_RatJ = True  # RatJDay2 has < 15 neurons which meets our exclusion criteria
if drop_RatJ:
    frate_df = frate_df[frate_df.name != "RatJDay1"]
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)
    if "0-2.5" in df.keys():
        value_vars = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    else: 
        value_vars = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=value_vars,
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

drop_append = "_dropRatJ" if drop_RatJ else ""
subjects.GroupData().save(frate_bs_df, f"frate_blocks_bootstrap_session_neurons{drop_append}")

In [ ]:
frate_dfz = subjects.GroupData().frate_post_chunks_zscore_pre
drop_RatJ = True  # RatJDay2 has < 15 neurons which meets our exclusion criteria
if drop_RatJ:
    frate_dfz = frate_dfz[frate_dfz.name != "RatJDay1"]
frate_dfz = (
    frate_dfz.pivot_table(
        columns=["zt"],
        values=["frate_zscore_pre"],
        index=["grp", "session", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_dfz[frate_dfz.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    if "0-2.5" in df.keys():
        value_vars = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    else: 
        value_vars = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=value_vars,
        var_name="zt",
        value_name="frate_zscore_pre",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

drop_append = "_dropRatJ" if drop_RatJ else ""
subjects.GroupData().save(frate_bs_df, f"frate_blocks_bootstrap_session_neurons_zscore_pre{drop_append}")

In [ ]:
from plotters import Plotter

frate_df = subjects.GroupData().frate_blocks_bootstrap_session_neurons

fig = plotting.Fig(5, 5)
ax = fig.subplot(fig.gs[0])
plotter = Plotter(
    data=frate_df[frate_df.neuron_type == "inter"],
    x="zt",
    y="frate",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
)
plotter.violinplot_sd(palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_within=get_bootstrap_prob_paired,
    stat_across=get_bootstrap_prob,
    alpha_within=0.025,
    alpha_across=0.025,
)

### Firing rate pre + maze normalized

In [ ]:
frate_dfz2 = subjects.GroupData().frate_post_chunks_zscore_pre_maze
frate_dfz2 = (
    frate_dfz2.pivot_table(
        columns=["zt"],
        values=["frate_zscore_pre_maze"],
        index=["grp", "session", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_dfz2[frate_dfz2.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"],
        var_name="zt",
        value_name="frate_zscore_pre_maze",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_blocks_bootstrap_session_neurons_zscore_pre_maze")

### Firing rates normalize to MAZE

In [ ]:
frate_dfz3 = subjects.GroupData().frate_post_chunks_zscore_maze
frate_dfz3 = (
    frate_dfz3.pivot_table(
        columns=["zt"],
        values=["frate_zscore_maze"],
        index=["grp", "session", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_dfz2[frate_dfz2.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    if "0-2.5" in df.keys():
        value_vars = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    else: 
        value_vars = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=value_vars,
        var_name="zt",
        value_name="frate_zscore_maze",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_blocks_bootstrap_session_neurons_zscore_maze")

### Firing rates divided by baseline (pre) - Vyazovskiy method

In [ ]:
frate_change_df = subjects.GroupData().frate_change_pre_to_post
drop_RatJ = True  # RatJDay2 has < 15 neurons which meets our exclusion criteria
if drop_RatJ:
    frate_change_df = frate_change_df[frate_change_df.name != "RatJDay1"]
frate_change_df = (
    frate_change_df.pivot_table(
        columns=["zt"],
        values=["frate/mean_frate(PRE)"],
        index=["grp", "session", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_change_df[frate_change_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    if "0-2.5" in df.keys():
        value_vars = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    else: 
        value_vars = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=value_vars,
        var_name="zt",
        value_name="frate/mean_frate(PRE)",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

drop_append = "_dropRatJ" if drop_RatJ else ""
subjects.GroupData().save(frate_bs_df, f"frate_blocks_bootstrap_session_neurons_change_pre_to_post{drop_append}")

### Log Firing rates

In [ ]:
frate_dflog = subjects.GroupData().frate_log_post_chunks
frate_dflog = (
    frate_dflog.pivot_table(
        columns=["zt"],
        values=["log(frate)"],
        index=["grp", "session", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_dflog[frate_dflog.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"],
        var_name="zt",
        value_name="log(frate)",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_blocks_bootstrap_session_neurons_log")

### (1h block) Firing rate bootstrap means

In [ ]:
frate_df = subjects.GroupData().frate_1h_blocks
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-1", "4-5", "5-6"],
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

# subjects.GroupData().save(frate_bs_df, "frate_1h_blocks_bootstrap")

### Firing rate within ripples

In [ ]:
frate_df = subjects.GroupData().frate_in_ripple
frate_df = frate_df[frate_df["where"] == "inside"].drop(columns="where")
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = lambda df: df.drop("session", axis=1).groupby(["grp"]).mean().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"],
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_in_ripple_blocks_bootstrap_session_neurons")

In [ ]:
from plotters import Plotter

frate_df = subjects.GroupData().frate_blocks_bootstrap_session_neurons

fig = plotting.Fig(5, 5)
ax = fig.subplot(fig.gs[0])
plotter = Plotter(
    data=frate_df[frate_df.neuron_type == "inter"],
    x="zt",
    y="frate",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
)
plotter.violinplot_sd(palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_within=get_bootstrap_prob_paired,
    stat_across=get_bootstrap_prob,
    alpha_within=0.025,
    alpha_across=0.025,
)

### Firing rate IQR bootstrap values
- To compare distribution of firing rates using bootstrap

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = (
    lambda df: df.drop("session", axis=1)
    .groupby(["grp"])
    .quantile([0.25, 0.75])
    .diff()
    .iloc[[1]]
    .reset_index()
)


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"],
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

# subjects.GroupData().save(frate_bs_df, "frate_IQR_blocks_bootstrap_session_neurons")

In [ ]:
from plotters import Plotter

frate_df = subjects.GroupData().frate_IQR_blocks_bootstrap_session_neurons

fig = plotting.Fig(5, 5)
ax = fig.subplot(fig.gs[0])
plotter = Plotter(
    data=frate_df[frate_df.neuron_type == "pyr"],
    x="zt",
    y="frate",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
)
plotter.violinplot_sd(palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_within=get_bootstrap_prob_paired,
    stat_across=get_bootstrap_prob,
    alpha_within=0.025,
    alpha_across=0.025,
)

### Firing rate standard deviation bootstrap values
- To compare distribution of firing rates using bootstrap

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = lambda df: df.drop("session", axis=1).kurtosis().reset_index()


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"],
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_Kurtosis_blocks_bootstrap_session_neurons")

In [ ]:
from plotters import Plotter

frate_df = subjects.GroupData().frate_STD_blocks_bootstrap_session_neurons

fig = plotting.Fig(5, 5)
ax = fig.subplot(fig.gs[0])
plotter = Plotter(
    data=frate_df[frate_df.neuron_type == "pyr"],
    x="zt",
    y="frate",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
)
plotter.violinplot_sd(palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_within=get_bootstrap_prob_paired,
    stat_across=get_bootstrap_prob,
    alpha_within=0.025,
    alpha_across=0.025,
)

### Firing rate bootstrap means statewise

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks_statewise

drop_session13pre = False  # See sd_fig2_supp - session 13 shouldn't be included for NREM PRE (RatUDay4SD)
if drop_session13pre:  # Set session 13 firing rates to nan
    frate_df.loc[(frate_df.session == 13) & (frate_df.brainstate == "NREM") & (frate_df.zt == "PRE"), "frate"] = np.nan
    save_append = "_dropsession13"
else:
    save_append = ""
    
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type", "brainstate"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = (
    lambda df: df.drop("session", axis=1)
    .groupby(["grp"])
    .mean(numeric_only=True)
    .reset_index()
)

for state_name in ["WK", "NREM"]:
    state_df = frate_df[frate_df.brainstate == state_name]
    frate_bs_df = []
    n_iter = 10000 if state_name == "WK" else 12000  # grab extra bootstraps to account for missing a few due to some sessions have no neurons in NREM
    for n, ntype in enumerate(["pyr", "inter"]):
        ntype_df = state_df[state_df.neuron_type == ntype].drop("neuron_type", axis=1)
        df = bootstrap_resample(
            ntype_df, level="both", n_iter=n_iter, n_jobs=6, apply=func,
        )

        epoch_names = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
        if "ZT 0-2.5" in df.keys():
            epoch_names = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
            
        df = pd.melt(
            df,
            id_vars=["grp"],
            value_vars=epoch_names,
            var_name="zt",
            value_name="frate",
        )
        df["neuron_type"] = ntype

        # drop any rows from SD NREM with nans and then resample
        df_sub = df[((df.grp == "SD") & ((df.zt == "PRE") | (df.zt == "ZT 2.5-5"))) | ((df.grp == "NSD") & (df.zt == "PRE"))]  # these are the sessions which can have 0 neurons
        nan_bool = np.isnan(df_sub.frate)
        ind_drop = df_sub.index[nan_bool]  # Indices in df to drop
        df = df.drop(labels=df_sub.index[nan_bool], axis=0)  # drop nan indices
        # now resample 10000 bootstraps!
        df["zt"] = pd.Categorical(df["zt"], ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"])  # Make zt categorical to preserve epoch order
        df = df.groupby(["grp", "zt", "neuron_type"], sort="zt").sample(10000).reset_index()
        
        frate_bs_df.append(df)

    frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)
    subjects.GroupData().save(
        frate_bs_df, f"frate_blocks_{state_name}_bootstrap_session_neurons{save_append}"
    )

In [ ]:
from plotters import Plotter

frate_df = subjects.GroupData().frate_blocks_wk_bootstrap_session_neurons

fig = plotting.Fig(5, 5)
ax = fig.subplot(fig.gs[0])
plotter = Plotter(
    data=frate_df[frate_df.neuron_type == "inter"],
    x="zt",
    y="frate",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
)
plotter.violinplot_sd(palette=subjects.colors_sd(1)).stat_anot_sd(
    stat_within=get_bootstrap_prob_paired,
    stat_across=get_bootstrap_prob,
    alpha_within=[0.025, 0.005, 0.0005],
    alpha_across=[0.05, 0.01, 0.001],
    verbose=True,
)

fig.savefig(subjects.figpath_sd / "frate_inter_wk")

### Firing rates QW vs AW

In [2]:
frate_df = subjects.GroupData().frate_post_chunks_qw_aw

# drop_session13pre = False  # See sd_fig2_supp - session 13 shouldn't be included for NREM PRE (RatUDay4SD)
# if drop_session13pre:  # Set session 13 firing rates to nan
#     frate_df.loc[(frate_df.session == 13) & (frate_df.brainstate == "NREM") & (frate_df.zt == "PRE"), "frate"] = np.nan
#     save_append = "_dropsession13"
# else:
#     save_append = ""
    
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type", "brainstate"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = (
    lambda df: df.drop("session", axis=1)
    .groupby(["grp"])
    .mean(numeric_only=True)
    .reset_index()
)

for state_name in ["QW", "AW"]:
    state_df = frate_df[frate_df.brainstate == state_name]
    frate_bs_df = []
    n_iter = 10000
    # n_iter = 10000 if state_name == "WK" else 12000  # grab extra bootstraps to account for missing a few due to some sessions have no neurons in NREM
    for n, ntype in enumerate(["pyr", "inter"]):
        ntype_df = state_df[state_df.neuron_type == ntype].drop("neuron_type", axis=1)
        df = bootstrap_resample(
            ntype_df, level="both", n_iter=n_iter, n_jobs=6, apply=func,
        )

        epoch_names = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
        if "ZT 0-2.5" in df.keys():
            epoch_names = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
            
        df = pd.melt(
            df,
            id_vars=["grp"],
            value_vars=epoch_names,
            var_name="zt",
            value_name="frate",
        )
        df["neuron_type"] = ntype

        # drop any rows from SD NREM with nans and then resample
        df_sub = df[((df.grp == "SD") & ((df.zt == "PRE") | (df.zt == "ZT 2.5-5"))) | ((df.grp == "NSD") & (df.zt == "PRE"))]  # these are the sessions which can have 0 neurons
        nan_bool = np.isnan(df_sub.frate)
        ind_drop = df_sub.index[nan_bool]  # Indices in df to drop
        df = df.drop(labels=df_sub.index[nan_bool], axis=0)  # drop nan indices
        # now resample 10000 bootstraps!
        df["zt"] = pd.Categorical(df["zt"], ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"])  # Make zt categorical to preserve epoch order
        df = df.groupby(["grp", "zt", "neuron_type"], sort="zt").sample(10000).reset_index()
        
        frate_bs_df.append(df)

    frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)
    subjects.GroupData().save(
        frate_bs_df, f"frate_blocks_{state_name}_bootstrap_session_neurons"
    )

Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 931.05it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 905.10it/s]


Running bootstraps for NSD group


...

Running bootstraps for NSD group


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:09<00:00, 1013.67it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 905.47it/s]


frate_blocks_AW_bootstrap_session_neurons saved


### Firing Rates AW vs QW (low-delta) vs QW (high-delta)

In [9]:
# Compare AW vs QW split above to AW vs QW with hi/lo delta below. Great! Difference between the two means is exactly 0.0!

frate_df_split = subjects.GroupData().frate_post_chunks_qw_delta_split_aw
frate_df_split[frate_df_split.brainstate == "AW"].groupby(["grp", "neuron_type", "zt"]).mean(numeric_only=True)

frate_df = subjects.GroupData().frate_post_chunks_qw_aw
frate_df_split[frate_df_split.brainstate == "AW"].groupby(["grp", "neuron_type", "zt"]).mean(numeric_only=True) - frate_df[frate_df.brainstate == "AW"].groupby(["grp", "neuron_type", "zt"]).mean(numeric_only=True)

session  neuron_id  frate
grp neuron_type zt                                 
NSD inter       MAZE          0.0        0.0    0.0
                PRE           0.0        0.0    0.0
                ZT 0-2.5      0.0        0.0    0.0
                ZT 2.5-5      0.0        0.0    0.0
                ZT 5-7.5      0.0        0.0    0.0
    pyr         MAZE          0.0        0.0    0.0
                PRE           0.0        0.0    0.0
                ZT 0-2.5      0.0        0.0    0.0
                ZT 2.5-5      0.0        0.0    0.0
                ZT 5-7.5      0.0        0.0    0.0
SD  inter       MAZE          0.0        0.0    0.0
                PRE           0.0        0.0    0.0
                ZT 0-2.5      0.0        0.0    0.0
                ZT 2.5-5      0.0        0.0    0.0
                ZT 5-7.5      0.0        0.0    0.0
    pyr         MAZE          0.0        0.0    0.0
                PRE           0.0        0.0    0.0
                ZT 0-2.5      0.0        0.0    0.0
                ZT 2.5-5      0.0        0.0    0.0
                ZT 5-7.5      0.0        0.0    0.0

In [10]:
frate_df = subjects.GroupData().frate_post_chunks_qw_delta_split_aw

# drop_session13pre = False  # See sd_fig2_supp - session 13 shouldn't be included for NREM PRE (RatUDay4SD)
# if drop_session13pre:  # Set session 13 firing rates to nan
#     frate_df.loc[(frate_df.session == 13) & (frate_df.brainstate == "NREM") & (frate_df.zt == "PRE"), "frate"] = np.nan
#     save_append = "_dropsession13"
# else:
#     save_append = ""
    
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type", "brainstate"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = (
    lambda df: df.drop("session", axis=1)
    .groupby(["grp"])
    .mean(numeric_only=True)
    .reset_index()
)

for state_name in ['QW_lowdelta', 'QW_highdelta']:  # No need to re-do AW since it is exactly the same as the above
    state_df = frate_df[frate_df.brainstate == state_name]
    frate_bs_df = []
    n_iter = 10000
    # n_iter = 10000 if state_name == "WK" else 12000  # grab extra bootstraps to account for missing a few due to some sessions have no neurons in NREM
    for n, ntype in enumerate(["pyr", "inter"]):
        ntype_df = state_df[state_df.neuron_type == ntype].drop("neuron_type", axis=1)
        df = bootstrap_resample(
            ntype_df, level="both", n_iter=n_iter, n_jobs=6, apply=func,
        )

        epoch_names = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
        if "ZT 0-2.5" in df.keys():
            epoch_names = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
            
        df = pd.melt(
            df,
            id_vars=["grp"],
            value_vars=epoch_names,
            var_name="zt",
            value_name="frate",
        )
        df["neuron_type"] = ntype

        # drop any rows from SD NREM with nans and then resample
        df_sub = df[((df.grp == "SD") & ((df.zt == "PRE") | (df.zt == "ZT 2.5-5"))) | ((df.grp == "NSD") & (df.zt == "PRE"))]  # these are the sessions which can have 0 neurons
        nan_bool = np.isnan(df_sub.frate)
        ind_drop = df_sub.index[nan_bool]  # Indices in df to drop
        df = df.drop(labels=df_sub.index[nan_bool], axis=0)  # drop nan indices
        # now resample 10000 bootstraps!
        df["zt"] = pd.Categorical(df["zt"], ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"])  # Make zt categorical to preserve epoch order
        df = df.groupby(["grp", "zt", "neuron_type"], sort="zt").sample(10000).reset_index()
        
        frate_bs_df.append(df)

    frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)
    subjects.GroupData().save(
        frate_bs_df, f"frate_blocks_{state_name}_bootstrap_session_neurons"
    )

Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 823.21it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 884.38it/s]


Running bootstraps for NSD group


...

Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 988.23it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 897.88it/s]


frate_blocks_QW_highdelta_bootstrap_session_neurons saved


### Bootstrap values of test statistic for normality tests
We reported in the main text of Giri2023 et al.
> Differences were also evident in the distributions of pyramidal cell firing rate, which
were approximately log-normal during natural sleep, but were heavily skewed away from
the log-normal during sleep deprivation, with a broader distribution of firing rates compared
with natural sleep (NS2 IQR = 0.62 log(Hz), 𝑝𝑝 = 0.35, Shapiro-Wilk test (SWT) on log firing rates (df=442) vs SD2.

To do this using bootstrap, we decided to calculate Shapiro-Wilk test statistic for each bootstrap and compare their distributions between NSD and SD.

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
# frate_df = frate_df[frate_df["session"] != 0]


def func(df):
    columns = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    statistic = []
    for col in columns:
        statistic.append(
            [
                *stats.shapiro(np.log10(df.loc[:, col].values)),
                *stats.skewtest(np.log10(df.loc[:, col].values)),
                # stats.kurtosistest(np.log10(df.loc[:, col]))[0], # Warning for n<=20
                # stats.normaltest(np.log10(df.loc[:, col]))[0], # warning for n<=20
                stats.iqr(np.log10(df.loc[:, col].values)),
            ]
        )
    statistic = np.array(statistic)
    stat_df = pd.DataFrame(
        statistic, columns=["shapiro", "shapiro_pvalue", "skew", "skew_pvalue", "iqr"]
    )
    stat_df["zt"] = columns
    return stat_df
frate_df_pyr = frate_df[frate_df.neuron_type == "pyr"]
frate_stat_session_df_pyr = frate_df_pyr.groupby(["session", "grp"]).apply(func).reset_index()
# subjects.GroupData().save(frate_stat_session_df, "frate_normality_statistic_blocks")
subjects.GroupData().save(frate_stat_session_df_pyr, "frate_normality_statistic_blocks_pyr_only")

frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_normality_statistic_blocks_bootstrap")

In [ ]:
# Display counts of int vs pyr for each session.
frate_df.groupby(["session", "grp", "neuron_type"]).count()

#### Same as above but statewise: NREM vs WAKE

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks_statewise
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "brainstate", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
frate_df = frate_df.loc[(frate_df!=0).all(axis=1)] # Drop any cells that have a 0 firing rate at any point
# frate_df = frate_df[frate_df["session"] != 0]


def func(df):
    columns = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    statistic = []
    for col in columns:
        statistic.append(
            [
                *stats.shapiro(np.log10(df.loc[:, col].values)),
                *stats.skewtest(np.log10(df.loc[:, col].values)),
                # stats.kurtosistest(np.log10(df.loc[:, col]))[0], # Warning for n<=20
                # stats.normaltest(np.log10(df.loc[:, col]))[0], # warning for n<=20
                stats.iqr(np.log10(df.loc[:, col].values)),
            ]
        )
    statistic = np.array(statistic)
    stat_df = pd.DataFrame(
        statistic, columns=["shapiro", "shapiro_pvalue", "skew", "skew_pvalue", "iqr"]
    )
    stat_df["zt"] = columns
    return stat_df
frate_df_pyr = frate_df[frate_df.neuron_type == "pyr"]
frate_stat_session_df_pyr = frate_df_pyr.groupby(["grp", "brainstate", "session"]).apply(func).reset_index()
# subjects.GroupData().save(frate_stat_session_df, "frate_normality_statistic_blocks_statewise")
subjects.GroupData().save(frate_stat_session_df_pyr, "frate_normality_statistic_blocks_pyr_only_statewise")

frate_bs_df = []
for st in ["WK", "NREM"]:
    for n, ntype in enumerate(["pyr", "inter"]):
        print(f"{st} {ntype} bootstrapping")
        ntype_df = frate_df[(frate_df.neuron_type == ntype) & (frate_df.brainstate == st)].drop("neuron_type", axis=1)
        df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)
        df["neuron_type"] = ntype
        df["brainstate"] = st
        frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_normality_statistic_blocks_bootstrap_statewise")

#### Same as above but for QW vs AW

In [4]:
frate_df = subjects.GroupData().frate_post_chunks_qw_aw
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "brainstate", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
frate_df = frate_df.loc[(frate_df!=0).all(axis=1)] # Drop any cells that have a 0 firing rate at any point
# frate_df = frate_df[frate_df["session"] != 0]


def func(df):
    columns = ["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"]
    statistic = []
    for col in columns:
        statistic.append(
            [
                *stats.shapiro(np.log10(df.loc[:, col].values)),
                *stats.skewtest(np.log10(df.loc[:, col].values)),
                # stats.kurtosistest(np.log10(df.loc[:, col]))[0], # Warning for n<=20
                # stats.normaltest(np.log10(df.loc[:, col]))[0], # warning for n<=20
                stats.iqr(np.log10(df.loc[:, col].values)),
            ]
        )
    statistic = np.array(statistic)
    stat_df = pd.DataFrame(
        statistic, columns=["shapiro", "shapiro_pvalue", "skew", "skew_pvalue", "iqr"]
    )
    stat_df["zt"] = columns
    return stat_df
frate_df_pyr = frate_df[frate_df.neuron_type == "pyr"]
frate_stat_session_df_pyr = frate_df_pyr.groupby(["grp", "brainstate", "session"]).apply(func).reset_index()
# subjects.GroupData().save(frate_stat_session_df, "frate_normality_statistic_blocks_statewise")
subjects.GroupData().save(frate_stat_session_df_pyr, "frate_normality_statistic_blocks_pyr_only_qw_aw")

frate_bs_df = []
for st in ["QW", "AW"]:
    for n, ntype in enumerate(["pyr", "inter"]):
        print(f"{st} {ntype} bootstrapping")
        ntype_df = frate_df[(frate_df.neuron_type == ntype) & (frate_df.brainstate == st)].drop("neuron_type", axis=1)
        df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)
        df["neuron_type"] = ntype
        df["brainstate"] = st
        frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_normality_statistic_blocks_bootstrap_qw_aw")

frate_normality_statistic_blocks_pyr_only_qw_aw saved
QW pyr bootstrapping
Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:18<00:00, 539.53it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:19<00:00, 510.46it/s]


QW inter bootstrapping
Running bootstraps for NSD group


...

AW inter bootstrapping
Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:17<00:00, 580.42it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:19<00:00, 523.37it/s]


frate_normality_statistic_blocks_bootstrap_qw_aw saved


In [6]:
frate_df = subjects.GroupData().frate_post_chunks_qw_aw
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "brainstate", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)

func = (
    lambda df: df.drop("session", axis=1)
    .groupby(["grp", "brainstate"])
    .quantile([0.25, 0.75])
    .diff()
    .iloc[[1]]
    .reset_index()
)


frate_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_df[frate_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)

    df = pd.melt(
        df,
        id_vars=["grp"],
        value_vars=["PRE", "MAZE", "ZT 0-2.5", "ZT 2.5-5", "ZT 5-7.5"],
        var_name="zt",
        value_name="frate",
    )
    df["neuron_type"] = ntype
    frate_bs_df.append(df)

frate_bs_df = pd.concat(frate_bs_df, ignore_index=True)

subjects.GroupData().save(frate_bs_df, "frate_IQR_blocks_bootstrap_session_neurons_qw_aw")

Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:13<00:00, 767.58it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:13<00:00, 725.60it/s]


Running bootstraps for NSD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 829.16it/s]


Running bootstraps for SD group


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 770.28it/s]


frate_IQR_blocks_bootstrap_session_neurons_qw_aw saved


#### FR distribution tests within ripples

In [ ]:
frate_df = subjects.GroupData().frate_post_chunks
frate_df = (
    frate_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
frate_df
# frate_df = frate_df[frate_df["session"] != 0]

In [ ]:
frate_rip_df = subjects.GroupData().frate_in_ripple
frate_rip_df = frate_rip_df[frate_rip_df["where"] == "inside"]
frate_rip_df = (
    frate_rip_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
frate_rip_df

In [ ]:
frate_rip_df = subjects.GroupData().frate_in_ripple
frate_rip_df = frate_rip_df[frate_rip_df["where"] == "inside"]
frate_rip_df = frate_rip_df.loc[(frate_rip_df!=0).all(axis=1)] # Drop any cells that have a 0 firing rate at any point
frate_rip_df = (
    frate_rip_df.pivot_table(
        columns=["zt"],
        values=["frate"],
        index=["grp", "session", "neuron_id", "neuron_type"],
        sort=False,
    )
    .reset_index(col_level=1)
    .droplevel(0, axis=1)
    .drop(["neuron_id"], axis=1)
)
frate_rip_df

def func(df):
    columns = ["PRE", "MAZE", "0-2.5", "2.5-5", "5-7.5"]
    statistic = []
    for col in columns:
        statistic.append(
            [
                *stats.shapiro(np.log10(df.loc[:, col].values)),
                *stats.skewtest(np.log10(df.loc[:, col].values)),
                # stats.kurtosistest(np.log10(df.loc[:, col]))[0], # Warning for n<=20
                # stats.normaltest(np.log10(df.loc[:, col]))[0], # warning for n<=20
                stats.iqr(np.log10(df.loc[:, col].values)),
            ]
        )
    statistic = np.array(statistic)
    stat_df = pd.DataFrame(
        statistic, columns=["shapiro", "shapiro_pvalue", "skew", "skew_pvalue", "iqr"]
    )
    stat_df["zt"] = columns
    return stat_df

frate_rip_bs_df = []
for n, ntype in enumerate(["pyr", "inter"]):
    ntype_df = frate_rip_df[frate_rip_df.neuron_type == ntype].drop("neuron_type", axis=1)
    df = bootstrap_resample(ntype_df, level="both", n_iter=10000, n_jobs=6, apply=func)
    df["neuron_type"] = ntype
    frate_rip_bs_df.append(df)

frate_rip_bs_df = pd.concat(frate_rip_bs_df, ignore_index=True)

subjects.GroupData().save(frate_rip_bs_df, "frate_win_ripple_normality_statistic_blocks_bootstrap")